In [1]:
# AI for self Driving car

In [2]:
# Importing the libraries

import numpy as np
import torch
import os
import random
import torch.nn as nn # All the essential tools
import torch.nn.functional as F 
import torch.optim as optim
import torch.autograd as autogra
from torch.autograd import Variable

In [3]:
# Creating the architecture of the Neural Network
# five neurons for 5 dimensions for input, 3 output neurons

class Network(nn.Module):
    
    def __init__(self, input_size, nb_action): # input_size = number of input neurons
        super(Network, self).__init__()       # nb_acton = no. of output neurons
        self.input_size = input_size
        self.nb_action = nb_action
        self.fc1 = nn.Linear(input_size, 30)  # 30 =  hidden layers
        self.fc2 = nn.Linear(30, nb_action)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        q_values = self.fc2(x)
        return q_values # returns all possible values forward, left, right

In [4]:
# Implementing Experience Replay
# AI process based on Markov Decision and its series of state

class ReplayMemory(object):
    
    def __init__(Self, capacity): #capacity simple less than 100 transaction
        self.capacity = capacity
        self.memory = []
        
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
            
    def sample(self, batch_size):
        samples = zip(*random.sample(self.memory, batch_size))  # function like reshape
        return map(lambda x: Variable(torch.cat(x, 0)), samples)

In [5]:
# Implementing Deep Q learning

class Dqn():
    
    def __init__(self, input_size, nb_action, gamma ):
        self.gamma = gamma
        self.reward_window = []
        self.model = Network(input_size, nb_action) # create neural network for perfect ldeep learning model
        self.memory = ReplayMemory(100000)
        self.optimizer = optim.Adam(self.model.parameters(), lr = 0.001)
        self.last_state = torch.Tensor(input_size).unsqueeze(0)
        self.last_action = 0
        self.last_reward = 0
        
    def select_action(self, state):
        probs = F.softmax(self.model(Variable(state, volatile = True)) * 0)
        action = probs.multinomial()
        return action.data[0,0]
    
    def learn(self, batch_state, batch_next_state, batch_reward, batch_action):
        outputs = self.model(batch_state) .gather(1, batch_action.unsqueeze(1)).squeeze(1)
        next_outputs = self.model(batch_next_state).detach().max(1)[0]
        target = self.gamma * next_outputs + batch_reward
        td_loss = F.smooth_l1_loss(outputs, target) #temporal difference
        self.optimizer.zero_grad()
        tf_loss.backward(retain_variables = True) #uses to update the weights
        self.optimizer.step()  # Update the weights
        
    def update(self, reward, new_signal):
        new_state = torch.Tensor(new_signal).float().unsqueeze(0)
        self.memory.push((self.last_state, new_state, torch.LongTensor([int(self.last_action)]),
                         torch.Tensor([self.last_reward])))
        action = self.select_action(new_state)
        if len(self.memory.memory) > 100:
            batch_state, batch_next_state, batch_action, batch_reward = self.memory.sample(100)
            self.learn(batch_state, batch_next_state, batch_reward, batch_action)
        self.last_Action = action
        self.last_state = new_state
        self.last_reward =  reward
        self.reward_window.append(reward)
        if len(self.reward_window) > 1000:
            del self.reward_window[0]
        return action
    
    def score(self):
        return sum(self.reward_window) / (len(self.reward_window) + 1.) # it should not be equal to 0 that's why added 1
    
    def save(self):
        torch.save({'state_dict' : self.model.state_dict(),
                    'optimizer' : self.optimizer.state_dict()},
                   'last_brain.pth')
        
    def load(self):
        if os.path.isfile('last_brain.pth'):
            print("=> Loading Checkpoint.. ")
            checkpoint = torch.load('last_brain.pth')
            self.model.load_state_dict(checkpoint['state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            print('done')
        else:
            print("no checkpoints found...")
        
        
    
    